In [ ]:
# default_exp codes

# CircuitPython_pico_pi_common.codes
> PPD Register & Command codes required for PPC classes.

In [ ]:
# export
from sys import byteorder

In [ ]:
# export

ID_CODE  = bytearray([ord(c) for c in list('ppdd')])
"""identifier string used by RPi(s) running ppdd, the PPC device daemon"""

REG_CODE = {
    'NOP': bytearray([  0]),      # no register, not used
    'CLR': bytearray([ord('F')]), # request to Flush/clear transmit FIFO
    'IDF': bytearray([ord('I')]), # request to send [str]  ID_CODE
    'BOS': bytearray([ord('B')]), # request to send [bool] Bosmang status
    'TIM': bytearray([ord('T')]), # request to send [int]  dateTime
    'CMD': bytearray([ord('C')]), # request to send [int]  Command
    'HOS': bytearray([ord('H')]), # request to send [str]  Hostname
    'LOD': bytearray([ord('L')]), # request to send [int]  Load
    'UPT': bytearray([ord('U')]), # request to send [int]  Uptime
    'TZN': bytearray([ord('Z')]), # request to send [int]  timeZone (sec offset from UTC)
    'PEN': bytearray([ord('P')]), # request to send [int]  MCU pin connected to RPi PEN
    
    'CFM': bytearray([ord('A')]), # command to confirm [int+int] Confirm/Acknowledge last CMD
    'FLK': bytearray([ord('K')]), # command to flicker [int] PWR LED for seconds [int]
    'UID': bytearray([ord('V')]), # command to receive [int+bytearray] PPC len,microcontroller.cpu.uid
    'ICS': bytearray([ord('2')]), # command to receive [int+str] PPC len,I2C_str
    'MSG': bytearray([ord('M')]), # command to receive [int+str] message for display
    'REG': bytearray([ord('R')]), # command to receive [int+int+variable_len] PPD addr, REG_CODE, value
    'NAM': bytearray([ord('N')]), # command to receive [int+int+str] PPD addr,hostname (shorter than REG)
    'RPT': bytearray([ord('S')]), # command to receive [int] stats report data for N PPDs
    'PPD': bytearray([ord('D')]), # command to receive [int+int+str] PPD report addr,len,pack?
                                  # or use convention of R but send stored value without reg query
    'DRG': bytearray([245]),      # command to go DEREGISTER
    'OFL': bytearray([245]),      # command to go OFFLINE
    'ONL': bytearray([246]),      # command to go ONLINE
    'RBT': bytearray([247]),      # command to REBOOT
    'SDN': bytearray([248]),      # command to SHUTDOWN
    'ONN': bytearray([249]),      # command to POWERON
    'OFF': bytearray([250])}      # command to POWEROFF 
"""I2C Register codes for PPDevices"""

REG_NAME = {}
REG_INT = []
for name, code in REG_CODE.items():
    REG_NAME[int.from_bytes(code, byteorder)] = name
    REG_INT.append(int.from_bytes(code, byteorder))

REG_VAL_LEN = {# in bytes for first (sometimes only) read/write; + len for followon read/write
    'NOP': 0 , # hardware transmit FIFO of RPi secondary I2C periph is 16 bytes
    'CLR': 16, # hardware transmit FIFO of RPi secondary I2C periph is 16 bytes
    'IDF': len(ID_CODE),
    'BOS': 1,  # len bosmang bool
    'TIM': 4,  # len timestamp int
    'CMD': 1,  # len cmd_code + bytes in CMD_CODE
    'HOS': 1,  # len len(hostname) + len(hostname)
    'LOD': 4,  # len loadavg string from float
    'UPT': 4,  # len uptime seconds int
    'TZN': 3,  # len utcoffset seconds int
    'PEN': 1,  # len pin int of MCU GPIO connected to PIN
    
    'CFM': 2,  # len cmd_code + bytes in CMD_CODE
    'FLK': 2,  # len cmd_code + duration int
    'UID': 1,  # len ACK + echo: len len(UID) + len(UID); set in PPController instance __init__
    'ICS': 1,  # len ACK + echo: len len(ICS) + len(ICS); set in PPController instance __init__
    'MSG': 1,  # len ACK + echo: len len(msg) + len(msg)
    'REG': 2,  # len ACK + echo: REG_CODE + REG_VALUE_LEN + len(REG value); for given register for given PPD
    'NAM': 2,  # len ACK + echo: len(hostname) + len(hostname); for given PPD 
    'RPT': 1,  # len ACK + echo: len(ppds); ppds for which report data will be sent
    
    'DRG': 1,  # len ACK + echo: cmd_code + device_address
    'OFL': 1,  # len ACK + echo: cmd_code + device_address
    'ONL': 1,  # len ACK + echo: cmd_code + device_address
    'RBT': 1,  # len ACK + echo: cmd_code + device_address
    'SDN': 1,  # len ACK + echo: cmd_code + device_address
    'ONN': 1,  # len ACK + echo: cmd_code + device_address
    'OFF': 1 } # len ACK + echo: cmd_code + device_address
    
CMD_CATALOG = (
    #int, 'NAME',   bytes,  REGister map 
    (  0, 'NOP',        0, 'NOP'), # no command, not used
    ( 97, 'CONFIRM',    2, 'REG'), # device_address, cmd_code
    ( 99, 'OFFLINE',    1, 'OFL'), # device_address 
    ( 99, 'ONLINE',     1, 'ONL'), # device_address 
    (100, 'DEREGISTER', 1, 'DRG'), # device_address
    (101, 'REG_GET',    2, 'REG'), # device_address, reg_code ; used for:
          # hostname, bosmang, timezone, uart/gpio_poweroff/pen pins
          # PPDevice will update its self values.
          # ppdd must then accept incoming data addr+reg_code+value
          # 2nd followon CONFIRM send when complete
    (122, 'FLICKER',    2, 'FLK'), # device_address, duration 

    (226, 'ROUNDROBIN', 1, 'FLK'), # duration ; ALL PPDs
    (227, 'REPORT',     1, 'RPT'), # number of ppds ; 0xFF for all; if not 0xFF,
          # PPC then probes NAM register N times for list of addrs, 
          # hostname included for error detection.
    (247, 'REBOOT',     1, 'RBT'), # device_address 
    (248, 'SHUTDOWN',   1, 'SDN'), # device_address 
    (249, 'POWERON',    1, 'ONN'), # device_address 
    (250, 'POWEROFF',   1, 'OFF')) # device_address 
"""
Command cmd_code int values, command NAME, number of bytes remaining in command,
(plus the 64-bit (8 byte) unique identifier). 

All commands require followon CONFIRM.
All commands sending device_address may send 0xFF for ALL devices.
So as to avoid collisions with ASCII/UTF-8 control characters & capital letters
used by REG_CODE, purely to  avoid confusion, valid ranges are 97-122, 225-250.
Common commands (cmd_code <128) may be used by regular PPDs, but only on self.
Reserved commands (cmd_code >127) can be used externally only by bosmang.
Suggested for PPD to send OFFLNE before manual or programmed shutdown.
The number of PPDs that can be included in 'all' for POWERON & POWEROFF is
limited by the number of available MCU GPIOs for RPi-connected pins for
gpio_poweroff & PEN.
"""

COMPLEX_CMD = [101,226,227]
"""Commands that inherently work with multiple PPDs"""

CMD_CODE =    { }
CMD_NAME =    { }
CMD_ARG_LEN = { }
CMD_REG_MAP = { }
CMD_INT     = [ ]

for code, name, vallen, regmap in CMD_CATALOG:
    CMD_CODE[name]=code
    CMD_NAME[code]=name
    CMD_ARG_LEN[code]=vallen
    CMD_REG_MAP[code]=regmap
    CMD_INT.append(code)
 
REP_CLI_CATALOG = (
    ('hostname', 'hostname'),
    ('mcu_uid', 'uid'),
    ('ppc_i2c_str', 'id'),
    ('device_address', 'address'),
    ('lastonline', 'lastonline'),
    ('loadavg', 'loadavg'),
    ('bosmang', 'bosmang'),
    ('utcoffset', 'timezone')
)

def parse_cmd(command):
    cmd_code = None
    i2c_addr = None
    cmdargs  = None
    cmd_uid  = None
    valid_status = False
    
    try:
        cmd_code = command[0]
    except IndexError:
        return cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status
    
    if cmd_code in CMD_NAME.keys():
        try:
            i2c_addr = command[1]  #only 1 byte pulled, result is int
        except IndexError:
            return cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status
        try:
            cmdargs = command[2:CMD_ARG_LEN[cmd_code]+1]
        except IndexError:
            return cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status
        try:
            cmd_uid = command[  CMD_ARG_LEN[cmd_code]+1:]
        except IndexError:
            return cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status
        if len(cmd_uid) > 8:  #too many bytes
            return cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status
    else:
        return cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status
    
    valid_status = True 
    return cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status

In [ ]:
# hide
try:
    from IPython.display import display, Javascript
    display(Javascript('IPython.notebook.save_checkpoint();'))
    from time import sleep
    sleep(0.3)
    from nbdev.export import notebook2script
    notebook2script()
except ModuleNotFoundError:
    pass
    """CircuitPython kernel has no nbdev"""

!!echo -e "\x02\x04" | tee -a /dev/ttyACM0
%cp -v CircuitPython_pico_pi_common/codes.py /CIRCUITPY/lib/CircuitPython_pico_pi_common/
!!sleep (0.2)
!!echo -e "\x04" | tee -a /dev/ttyACM0
%pip install .

<IPython.core.display.Javascript object>

Converted 00_core.ipynb.
Converted 01_codes.ipynb.
Converted 99_code.ipynb.
Converted index.ipynb.
'CircuitPython_pico_pi_common/codes.py' -> '/CIRCUITPY/lib/CircuitPython_pico_pi_common/codes.py'
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Processing /home/david/notebooks/CircuitPython_pico_pi_common
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for CircuitPython-pico-pi-common: filename=CircuitPython_pico_pi_common-0.0.1-py3-none-any.whl size=7464 sha256=459c02f847288b4f1d0a5d6bd660e1595d91b997360c464cb18876e50d538279
  Stored in directory: /tmp/pip-ephem-wheel-cache-ibwhm8nl/wheels/4d/53/ea/6531bef1ee298aaeb710928a72208a0dc5085ac3af881689ab
Successfully built CircuitPython-pico-pi-common
  Attempting uninstall: CircuitPython-pico-pi-common
    Found existing installation: Circuit